In [ ]:
!pip install -q datasets transformers evaluate scikit-learn matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="unitary/toxic-bert")

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import os
from huggingface_hub import InferenceClient
from google.colab import userdata
token = userdata.get('HF_TOKEN')
client = InferenceClient(
    provider="auto",
    api_key=token
)


In [ ]:
#Importar dataset
import pandas as pd
from datasets import Dataset, concatenate_datasets

df = pd.read_csv("/content/drive/MyDrive/JBB_dataset/train2.csv")
df.head()
df["text"] = df["raw_text"]
ds_train = Dataset.from_pandas(df)



Dataset({
    features: ['label', 'raw_text', 'text'],
    num_rows: 202
})

## Treinamento da camada 1

In [ ]:
# Tokenizer e modelo (Toxic-BERT)

model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,                              # Queremos apenas 2 classes
    ignore_mismatched_sizes=True,              # Permite substituir a camada final de 6 por 2
    problem_type="single_label_classification"
)
def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

ds_train = ds_train.map(tokenize_fn, batched=True)

# removendo apenas a coluna "text" (mantemos raw_text)
ds_train = ds_train.remove_columns([c for c in ds_train.column_names if c not in ["raw_text", "label", "input_ids", "attention_mask", "token_type_ids"] and c != "label"])

# Converter para formato torch
ds_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/222 [00:00<?, ? examples/s]

'ds_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])\n'

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir="./toxbert_jbb",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=6,
    logging_steps=20,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3395022559.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
20,0.712900
40,0.662800
60,0.610100
80,0.449500
100,0.322200
120,0.183900
140,0.185500
160,0.105900


TrainOutput(global_step=168, training_loss=0.38958026184922173, metrics={'train_runtime': 80.2361, 'train_samples_per_second': 16.601, 'train_steps_per_second': 2.094, 'total_flos': 175231962869760.0, 'train_loss': 0.38958026184922173, 'epoch': 6.0})

In [ ]:
trainer.model.save_pretrained('/content/drive/MyDrive/JBB_dataset')

# Salvar o tokenizer (vocabulário e regras)
tokenizer.save_pretrained('/content/drive/MyDrive/JBB_dataset')

('/content/drive/MyDrive/JBB_dataset/tokenizer_config.json',
 '/content/drive/MyDrive/JBB_dataset/special_tokens_map.json',
 '/content/drive/MyDrive/JBB_dataset/vocab.txt',
 '/content/drive/MyDrive/JBB_dataset/added_tokens.json',
 '/content/drive/MyDrive/JBB_dataset/tokenizer.json')

In [ ]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify(text):
    model.eval()
    tokens = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="pt"
    ).to(device)  # <-- mover tokens para GPU
    print(text)
    with torch.no_grad():
        outputs = model(**tokens)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[0]

    pred = torch.argmax(probs).item()
    print(pred)

    prob = probs[pred].item()
    p_safe = probs[0].item()
    p_unsafe = probs[1].item()

    if p_unsafe >= 0.4:
        pred = 1
    else:
        pred = 0

    return {
        "label": "unsafe" if pred == 1 else "safe",
        "probability": prob,
        "raw_probs": probs.tolist()
    }

text= """cake and biscuits and pies and coking """
print(classify(text))

cake and biscuits and pies and coking 
0
{'label': 'safe', 'probability': 0.8950116038322449, 'raw_probs': [0.8950116038322449, 0.10498839616775513]}


In [ ]:
# @title Carregar modelo
from google.colab import drive

drive.mount('/content/drive')

LOAD_PATH = "/content/drive/MyDrive/JBB_MODEL"

# 1. Carregar o Tokenizer
reloaded_tokenizer = AutoTokenizer.from_pretrained(LOAD_PATH)

# 2. Carregar o Modelo (com os pesos treinados)
reloaded_model = AutoModelForSequenceClassification.from_pretrained(
    LOAD_PATH,
    num_labels=2,
)

# Colocar o modelo em modo de avaliação e mover para a GPU, se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reloaded_model.to(device)
reloaded_model.eval()

print("Modelo recarregado com sucesso e pronto para inferência.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Modelo recarregado com sucesso e pronto para inferência.
